In [ ]:
#installations - needs to run the first time.
!pip install firebase
!pip install requests beautifulsoup4
!pip install nltk
!pip install pandas
!pip install matplotlib
!pip install pillow

In [5]:
#installations - needs to run the first time above.

#imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import io
import base64
from datetime import datetime, timedelta
import random
from PIL import Image
import re
from nltk.stem import PorterStemmer
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML
import os
import sys
import contextlib
import requests
from collections import Counter, defaultdict
from google.colab import files
from firebase import firebase
FBconn = firebase.FirebaseApplication('https://cloud-e4313-default-rtdb.firebaseio.com/', None)

from bs4 import BeautifulSoup
from urllib.parse import urljoin
from pickle import STRING
import json




########################################################
########################################################
############~ Start index to FireBase DB ~##############

class IndexService:
    def __init__(self):
        self.documents = {}
        self.index = {}
        self.visited_urls = set()  # Track visited URLs to avoid duplicates
        self.max_links = 5  # Maximum number of links to follow from each page

    #Index a site starting from start_url and following links up to max_depth
    def index_site(self, start_url, max_depth=5):
        if max_depth <= 0 or start_url in self.visited_urls:
            return

        # Add the URL to visited set
        self.visited_urls.add(start_url)

        # Index the current page
        soup = self.fetch_page(start_url)
        if soup is None:
            return

        # Index the current page's content
        self.index_page_content(soup, start_url)

        # Find links on the page
        links = []
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            # Create absolute URL
            full_url = urljoin(start_url, href)
            if full_url not in links:
              links.append(full_url)

        # Limit to max_links
        links = links[:self.max_links]

        # Follow links recursively
        for link in links:
            self.index_site(link, max_depth - 1)

    #Index the content of a page
    def index_page_content(self, soup, url):
      # Extract text and count words
      words = re.findall(r'[a-zA-Z]+', soup.get_text())  # return words
      words = [word.lower() for word in words]

      # Count word occurrences
      word_counts = dict(Counter(words))

      # Process the words
      word_counts = self.remove_stop_words(word_counts)
      word_counts = self.apply_stemming(word_counts)

      # Add to the index
      self.add_document(word_counts, url)

      return word_counts

    #Legacy method: Index a single page without following links
    def index_words(self, url):
        soup = self.fetch_page(url)
        if soup is None:
            return None

        return self.index_page_content(soup, url)

    #Add a document to the index
    def add_document(self, word_counts, url):
        # Store document content for reference
        doc_id = str(len(self.documents) + 1)
        self.documents[doc_id] = {
            'words': list(word_counts.keys()),
            'url': url
        }

        # Build the inverted index
        for word in word_counts.keys():
            if word not in self.index:
                self.index[word] = set()

            # Add the URL to the set for this word
            self.index[word].add(url)

        return self.documents[doc_id]

    #safe call for html page
    def fetch_page(self, url):
        try:
            # Send an HTTP GET request to the URL
            response = requests.get(url, timeout=10)
            #successful request
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                return soup #Return the parsed HTML (soup object)
            else:
                print(f"Failed to fetch {url}: Status code {response.status_code}")
                return None
        except Exception as e:
            # Handle any exceptions that may occur during the request
            print(f"Error fetching {url}: {e}")
            return None


    def remove_stop_words(self, index):
        try:
            # reading stop words file from github.
            response = requests.get("https://raw.githubusercontent.com/einavbs1/HW2_Cloud/main/stopwords_en.txt")
            response.raise_for_status()  # ensuring there is not error
            stop_words = response.text.splitlines()
        except Exception as e:
            print(f"Error while reading stop words {e}")
            # deault stop words if file isnt avaiable
            stop_words = ['the', 'is', 'at', 'which', 'on', 'a']
        return {word: count for word, count in index.items() if word.lower() not in stop_words}


    #stemiming index words, using porterstemmer
    def apply_stemming(self, index):
        stemmer = PorterStemmer()
        stemmed_index = {}
        for word, count in index.items():
            stemmed_word = stemmer.stem(word)
            if stemmed_word in stemmed_index:
                stemmed_index[stemmed_word] += count
            else:
                stemmed_index[stemmed_word] = count
        return stemmed_index

    #Convert index with sets to dictionary format for Firebase
    def convert_index_for_firebase(self):
        firebase_index = {}
        for word, urls in self.index.items():
            firebase_index[word] = list(urls)
        return firebase_index

    #Retrieve all data records
    def load_index_from_firebase(self):

        firebase_data = FBconn.get('/myMqttIndex/', None)
        #Load index from Firebase format back to Python sets
        self.index = {}
        for word, urls in firebase_data.items():
            self.index[word] = set(urls)
        return self.index

    def add_index_to_firebase(self):
      firebase_index = self.convert_index_for_firebase()
      FBconn.patch('/myMqttIndex/', firebase_index)



#############~ End index to FireBase DB ~#################
########################################################
########################################################


########################################################
########################################################
############~ Start Search Engine page ~################


image_url = "https://raw.githubusercontent.com/einavbs1/HW2_Cloud/main/Corelia_Logo_Final.png"

logo_widget = widgets.HTML(
    value=f'<img src="{image_url}" width="500" height="400">'
)



tabs = widgets.Tab()
# CSS Styling (General + Table)
style = """
<style>
    html, body {
        background-color: #f4f4f4;
        margin: 0;
        height: 100vh;
        display: flex;
        justify-content: center;
        align-items: center;
    }
    .main-container {
        background-color: #ffffff;
        border-radius: 16px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        padding: 30px;
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        width: 500px;
        min-height: 350px;
    }
    .search-input input {
        border-radius: 12px;
        border: 1px solid #ccc;
        padding: 10px;
        font-size: 16px;
        background-color: #f9f9f9;
        outline: none;
        width: 350px;
        transition: box-shadow 0.3s;
    }
    .search-input input:focus {
        box-shadow: 0 0 8px rgba(30, 144, 255, 0.5);
        border-color: #1E90FF;
    }
    .button-container {
        display: flex;
        gap: 15px;
        margin-top: 20px;
        margin-bottom: 20px;
    }
    .search-button, .lucky-button {
        display: flex;
        align-items: center;
        justify-content: center;
        text-align: center;
        font-size: 14px;
        border-radius: 8px;
        padding: 10px 20px;
        border: none;
        cursor: pointer;
        height: 40px;
        transition: background-color 0.3s;
    }
    .search-button {
        background-color: #009ACD;
        color: white;
        width: 130px;
    }
    .search-button:hover {
        background-color: #007C8A;
    }
    .lucky-button {
        background-color: #4CAF50;
        color: white;
        width: 160px;
    }
    .lucky-button:hover {
        background-color: #228B22;
    }
    .custom-label {
        color: red;
        font-weight: bold;
        margin-top: 15px;
    }
    /* Table Styling */
    .results-table {
        border-collapse: collapse;
        width: 500px;
        margin: 30px auto;
        font-family: Arial, sans-serif;
    }
    .results-table th, .results-table td {
        text-align: left;
        padding: 12px;
    }
    .results-table tr:nth-child(even) {
        background-color: #f2f2f2;
    }
    .results-table th {
        background-color: #4CAF50;
        color: white;
    }
    .results-table td a {
        color: #1E90FF;
        text-decoration: none;
    }
    .results-table td a:hover {
        text-decoration: underline;
    }
</style>
"""
display(widgets.HTML(value=style))
index_service = IndexService()

index_service.index_site("https://mqtt.org/")  # Perform the indexing on the site
index = index_service.convert_index_for_firebase()

# After performing indexing, upload the new index to Firebase
index_service.add_index_to_firebase()
tab1_content = widgets.Output()

#make the last tab toi unvisable if none results found.
def resetTabs():
    global tabs
    tabs.children=[tab0_content,tab1_content,tab2_content,tab3_content]
    children=[tab0_content,tab1_content,tab2_content,tab3_content]
    tabs.selected_index = 3


def readFile(fileName):
    with open(fileName, 'r', encoding="utf-8") as file:
        return file.read()


def remove_stop_words(query_list):
    try:
        # reading stop words file from github.
        response = requests.get("https://raw.githubusercontent.com/einavbs1/HW2_Cloud/main/stopwords_en.txt")
        response.raise_for_status()  # ensuring there is not error
        stop_words = response.text.splitlines()
    except Exception as e:
        print(f"Error while reading stop words {e}")
        # deault stop words if file isnt avaiable
        stop_words = ['the', 'is', 'at', 'which', 'on', 'a']
    return [word for word in query_list if word.lower() not in stop_words]

#steem words to root.
def apply_stemming(query_list):
    stemmer = PorterStemmer()
    return list(set(stemmer.stem(word).lower() for word in query_list))

#returns only the words thats in the index
def search(stemmed_query, index):
    results = {}
    for word in stemmed_query:
        if word in index:
            results[word] = index[word]
    return results

#This function is sorting the links from the most frequence to the last
def prioritize_pages(results):
    priority = {}
    for word, links in results.items():
        for link in links:
            priority[link] = priority.get(link, 0) + 1
    return dict(sorted(priority.items(), key=lambda item: item[1], reverse=True))

# UI Widgets
Query = widgets.Text(
    value='',
    placeholder='Enter your search...',
    layout=widgets.Layout(width='350px', height='40px')
)
Query.add_class("search-input")

searchBtn = widgets.Button(
    description="Search",
    layout=widgets.Layout(width='130px', height='40px')
)
searchBtn.add_class("search-button")

LuckyBtn = widgets.Button(
    description="I'm Feeling Lucky",
    layout=widgets.Layout(width='160px', height='40px')
)
LuckyBtn.add_class("lucky-button")

errMsg = widgets.Label(value='')
errMsg.add_class("custom-label")

searchResults = widgets.HTML(value='')

#getting query from the search tab and finding results
def get_search_results():
    if Query.value.strip() == '':
        errMsg.value = 'Please enter a query.'
        return {}
    errMsg.value = ''
    words = Query.value.split()
    filtered = remove_stop_words(words)
    stemmed = apply_stemming(filtered)
    results = search(stemmed, index)
    return prioritize_pages(results)

#if we needs to show results and the last tab is unvisable
def update_tabs():
    global tabs
    if len(tabs.children) < 5:
        tabs.children = [tab0_content,tab1_content,tab2_content, tab3_content, tab4_content]
        tabs.set_title(0, 'Admin')
        tabs.set_title(1, 'Graphs')
        tabs.set_title(2, 'Gamification')
        tabs.set_title(3, 'Search')
        tabs.set_title(4, 'Results')

#This function is openingh the results in the results last tab
def on_click_search(btn):
    results = get_search_results()
    if not results:
        errMsg.value = "No results found"
        resetTabs()
    else:
        update_tabs()
        table_html = "<table class='results-table'>"
        table_html += "<tr><th>#</th><th>Site</th><th>Link</th></tr>"
        for i, link in enumerate(results, start=1):
            link_url = f"https://{link}" if not link.startswith("http") else link
            domain = link.replace("https://", "").replace("http://", "")
            table_html += f"<tr><td>{i}</td><td>{domain}</td><td><a href='{link_url}' target='_blank'>Visit</a></td></tr>"
        table_html += "</table>"
        searchResults.value = table_html
        Query.value = ''
        # Refresh Tab 4 Content
        with tab4_content:
            tab4_content.clear_output()
            display(widgets.VBox([logo_widget, searchResults], layout=widgets.Layout(align_items='center')))

        tabs.selected_index = 4

#This function is openingh the best result in new tab and showing the link in the results last tab
def on_click_lucky(btn):
    # Get search results
    results = get_search_results()
    # If no results are found, display an error message and reset tabs
    if not results:
        errMsg.value = "No results found"
        resetTabs()
        return
    else:
      update_tabs()
      url = next(iter(results))
      # Ensure the URL starts with "http", if not, prepend "https://"
      link_url = f"https://{url}" if not url.startswith("http") else url
      display(Javascript(f"window.open('{link_url}', '_blank');"))
      searchResults.value = f"<p style='text-align:center; font-size: 18px;'>The best result is: <a href='{link_url}' target='_blank'>{link_url}</a><br>(Already opened for you.)</p>"
      Query.value = ''
      # Refresh Tab 4 Content
      with tab4_content:
            tab4_content.clear_output()
            display(widgets.VBox([logo_widget, searchResults], layout=widgets.Layout(align_items='center')))

      tabs.selected_index = 4


searchBtn.on_click(on_click_search)
LuckyBtn.on_click(on_click_lucky)
button_container = widgets.HBox([searchBtn, LuckyBtn])

# Create the main container for displaying the search interface, including logo, query, buttons, and error message
main_container = widgets.VBox(
    [logo_widget, Query, button_container, errMsg],
    layout=widgets.Layout(align_items='center', justify_content='center')
)

#############~ End Search Engine page ~#################
########################################################
########################################################

########################################################
########################################################
#################~ Start Graphs page ~##################


##################### Code from tut 03 that dosent works #####################
"""
מצרף את הקוד שמתבסס על תרגול 3 ללופ של הנתונים של החיישנים, בפועל זה לא עובד והאם קיוטיטי לא מפבלש הודעות לכן השתמשנו בפייק דאטה של נתונים שהצלחחתי לחלץ מהחיישנים מהזמן שכן פובלשו הודעות

import paho.mqtt.client as mqtt
import json

# Callback function for when a message is received
def on_message(client, userdata, msg):
    try:
        data = json.loads(msg.payload.decode())

        # Extract values with lowercase field names
        temperature = data.get("Temperature", "N/A")
        humidity = data.get("Humidity", "N/A")
        pressure = data.get("Pressure", "N/A")

        print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Pressure: {pressure} hPa")

    except json.JSONDecodeError:
        print("Received invalid JSON data")

# MQTT setup
broker = "test.mosquitto.org"
topic = "braude/D106/indoor"

client = mqtt.Client()
client.on_message = on_message

client.connect(broker, 1883, 60)
client.subscribe(topic)

print(f"Subscribed to MQTT topic: {topic}")
client.loop_forever()
"""

##################### End of the code from tut 03 that dosent works #####################




mpl.rcParams['font.family'] = 'DejaVu Sans'
plt.switch_backend('agg')

# === Generate fake data ===
def generate_fake_data():
    end_date = datetime.now()
    date_range = [end_date - timedelta(days=i) for i in range(6)]
    date_range.reverse()
    english_days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
    days_of_week = [english_days[d.weekday()] for d in date_range]

    # Existing sensor data
    avg_temps = [round(20 + random.random() * 6, 1) for _ in range(6)]
    avg_humidity = [round(40 + random.random() * 20, 1) for _ in range(6)]
    max_temps = [round(t + 2 + random.random() * 2, 1) for t in avg_temps]
    min_temps = [round(t - 2 - random.random() * 2, 1) for t in avg_temps]
    users = [round(10 + random.random() * 20) for _ in range(6)]
    avg_pressure = [round(1013 + random.uniform(-10, 10), 1) for _ in range(6)]

    # Daylight sensor data
    # Daylight hours per day (realistic range 8-14 hours depending on season/weather)
    daylight_hours = [round(random.uniform(9, 13), 1) for _ in range(6)]

    # Sunrise times (in hours, e.g., 6.5 = 6:30 AM)
    sunrise_times = [round(random.uniform(5.5, 7.0), 1) for _ in range(6)]

    # Sunset times (calculated from sunrise + daylight hours)
    sunset_times = [round(sunrise + daylight, 1) for sunrise, daylight in zip(sunrise_times, daylight_hours)]

    # Current daylight status
    current_hour = datetime.now().hour + datetime.now().minute / 60
    current_daylight_status = "Day" if 6 <= datetime.now().hour <= 18 else "Night"
    current_light_level = round(random.uniform(200, 1500), 0) if current_daylight_status == "Day" else round(random.uniform(0, 50), 0)

    # Current values for existing sensors
    current_temp = round(23.5 + (random.random() * 2 - 1), 1)
    current_humidity = round(48 + (random.random() * 8 - 4))
    current_active_users = 17 + (random.randint(-2, 2))
    current_pressure = round(1013 + random.uniform(-5, 5), 1)

    return {
        'days': days_of_week,
        'avg_temps': avg_temps,
        'max_temps': max_temps,
        'min_temps': min_temps,
        'avg_humidity': avg_humidity,
        'avg_pressure': avg_pressure,
        'users': users,
        # Daylight data
        'daylight_hours': daylight_hours,
        'sunrise_times': sunrise_times,
        'sunset_times': sunset_times,
        'current': {
            'temp': current_temp,
            'humidity': current_humidity,
            'pressure': current_pressure,
            'active_users': current_active_users,
            # Current daylight data
            'daylight_status': current_daylight_status,
            'light_level': current_light_level
        }
    }

def fig_to_img(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=150)
    buf.seek(0)
    img = Image.open(buf)
    plt.close(fig)  # Close figure to free memory
    return img

# === Existing Charts ===
def create_temp_humidity_chart(data):
    fig, ax1 = plt.subplots(figsize=(6, 3))
    ax1.set_xlabel('Day')
    ax1.set_ylabel('Temperature (°C)', color='salmon')
    plt.subplots_adjust(top=0.85)
    ax1.set_title('Average Daily Temperature Last 6 Days (Indoor Sensor)', pad=20)

    line1 = ax1.plot(data['days'], data['avg_temps'], color='salmon', marker='o', label='Temperature')
    ax1.tick_params(axis='y', labelcolor='salmon')

    ax2 = ax1.twinx()
    ax2.set_ylabel('Humidity (%)', color='turquoise')
    line2 = ax2.plot(data['days'], data['avg_humidity'], color='turquoise', marker='s', label='Humidity')
    ax2.tick_params(axis='y', labelcolor='turquoise')

    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, bbox_to_anchor=(0.02, 0.98), loc='upper left')

    fig.tight_layout()
    return fig_to_img(fig)

def create_pressure_chart(data):
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(data['days'], data['avg_pressure'], color='purple', marker='o')
    ax.axhline(y=1013.25, color='gray', linestyle='--', label='Standard Pressure (1013.25 hPa)')
    ax.set_ylabel('Pressure (hPa)')
    ax.set_title('Barometric Pressure - Last 6 Days (Indoor Sensor)')
    ax.legend()
    fig.tight_layout()
    return fig_to_img(fig)

def create_users_chart(data):
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.bar(data['days'], data['users'], color='cornflowerblue')
    ax.set_ylabel('Number of Users')
    ax.set_title('User Activity Per Day Of the Last week')
    fig.tight_layout()
    return fig_to_img(fig)

def create_temp_range_chart(data):
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.fill_between(data['days'], data['min_temps'], data['max_temps'], color='skyblue', alpha=0.3)
    ax.plot(data['days'], data['max_temps'], color='salmon', marker='o', label='Maximum Temperature')
    ax.plot(data['days'], data['min_temps'], color='deepskyblue', marker='o', label='Minimum Temperature')
    ax.set_ylabel('Temperature (°C)')
    ax.set_title('Min Max Temperature Per Day - Last 6 Days (Indoor Sensor)')
    ax.legend()
    fig.tight_layout()
    return fig_to_img(fig)

# === NEW: Daylight Charts ===
def create_daylight_hours_chart(data):
    fig, ax = plt.subplots(figsize=(6, 3))
    bars = ax.bar(data['days'], data['daylight_hours'], color='gold', alpha=0.8)
    ax.set_ylabel('Hours')
    ax.set_title('Daylight Hours Per Day - Last 6 Days (Outdoor Sensor)')
    ax.set_ylim(0, 15)

    # Add value labels on bars
    for bar, hours in zip(bars, data['daylight_hours']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{hours}h', ha='center', va='bottom', fontsize=10)

    fig.tight_layout()
    return fig_to_img(fig)

def create_sun_times_chart(data):
    fig, ax = plt.subplots(figsize=(6, 3))

    # Convert times to hours for plotting
    sunrise_hours = data['sunrise_times']
    sunset_hours = data['sunset_times']

    ax.plot(data['days'], sunrise_hours, color='orange', marker='o', label='Sunrise', linewidth=2)
    ax.plot(data['days'], sunset_hours, color='red', marker='s', label='Sunset', linewidth=2)

    # Fill area between sunrise and sunset to show daylight period
    ax.fill_between(data['days'], sunrise_hours, sunset_hours, color='yellow', alpha=0.2, label='Daylight Period')

    ax.set_ylabel('Time (24h format)')
    ax.set_title('Sunrise & Sunset Times - Last 6 Days (Outdoor Sensor)')
    ax.set_ylim(0, 24)

    # Format y-axis to show time format
    y_ticks = [0, 6, 12, 18, 24]
    y_labels = ['00:00', '06:00', '12:00', '18:00', '24:00']
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels)

    ax.legend()
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    return fig_to_img(fig)

# === Updated Info Cards ===
def create_info_html(current):
    return f"""
    <div style='display: flex; gap: 20px; justify-content: center; font-family: Arial; font-size: 16px; flex-wrap: wrap'>
        <div style='background: #fff; border-radius: 12px; padding: 16px; box-shadow: 0 0 10px rgba(0,0,0,0.1); width: 180px; text-align: center; margin: 10px'>
            <div style='font-size: 40px;'>🌡️</div>
            <strong style='color: black;'>Current Temperature</strong><br>
            <span style='color: black; font-size: 24px;'>{current['temp']}°C</span><br>
            <small style='color: black;'>Comfortable temperature</small>
        </div>
        <div style='background: #fff; border-radius: 12px; padding: 16px; box-shadow: 0 0 10px rgba(0,0,0,0.1); width: 180px; text-align: center; margin: 10px'>
            <div style='font-size: 40px;'>💧</div>
            <strong style='color: black;'>Current Humidity</strong><br>
            <span style='color: black; font-size: 24px;'>{current['humidity']}%</span><br>
            <small style='color: black;'>Comfortable humidity</small>
        </div>
        <div style='background: #fff; border-radius: 12px; padding: 16px; box-shadow: 0 0 10px rgba(0,0,0,0.1); width: 180px; text-align: center; margin: 10px'>
            <div style='font-size: 40px;'>🔄</div>
            <strong style='color: black;'>Current Pressure</strong><br>
            <span style='color: black; font-size: 24px;'>{current['pressure']} hPa</span><br>
            <small style='color: black;'>Normal pressure</small>
        </div>
        <div style='background: #fff; border-radius: 12px; padding: 16px; box-shadow: 0 0 10px rgba(0,0,0,0.1); width: 180px; text-align: center; margin: 10px'>
            <div style='font-size: 40px;'>👥</div>
            <strong style='color: black;'>Active Users</strong><br>
            <span style='color: black; font-size: 24px;'>{current['active_users']}</span><br>
            <small style='color: black;'>Normal activity</small>
        </div>
        <div style='background: #fff; border-radius: 12px; padding: 16px; box-shadow: 0 0 10px rgba(0,0,0,0.1); width: 180px; text-align: center; margin: 10px'>
            <div style='font-size: 40px;'>☀️</div>
            <strong style='color: black;'>Daylight Status</strong><br>
            <span style='color: black; font-size: 24px;'>{current['daylight_status']}</span><br>
            <small style='color: black;'>{current['light_level']} lux</small>
        </div>
    </div>
    """

# === Dashboard update ===
def update_dashboard():
    data = generate_fake_data()
    cards = create_info_html(data['current'])
    return (
        cards,
        create_temp_humidity_chart(data),
        create_pressure_chart(data),
        create_users_chart(data),
        create_temp_range_chart(data),
        create_daylight_hours_chart(data),
        create_sun_times_chart(data)
    )

def pil_to_widget_img(pil_img):
    buf = io.BytesIO()
    pil_img.save(buf, format='PNG')
    return widgets.Image(value=buf.getvalue(), format='png', width=500, height=300)

def create_demo():
    cards, temp_humidity_img, pressure_img, users_img, temp_range_img, daylight_hours_img, sun_times_img = update_dashboard()
    return widgets.VBox([
        widgets.HTML(value=cards),
        pil_to_widget_img(temp_humidity_img),
        pil_to_widget_img(pressure_img),
        pil_to_widget_img(users_img),
        pil_to_widget_img(temp_range_img),
        pil_to_widget_img(daylight_hours_img),
        pil_to_widget_img(sun_times_img)
    ], layout=widgets.Layout(align_items='center'))

refresh_button = widgets.Button(
    description="🔁 Refresh Data",
    layout=widgets.Layout(width='200px', height='40px', margin='0px 0px 40px 0px')
)

def refresh_dashboard(btn=None):
    tab1_content.clear_output(wait=True)
    with tab1_content:
        display(widgets.VBox(
            [logo_widget, refresh_button],
            layout=widgets.Layout(align_items='center', justify_content='center', gap='20px')
        ))

        # Load updated content including new daylight charts
        cards, temp_humidity_img, pressure_img, users_img, temp_range_img, daylight_hours_img, sun_times_img = update_dashboard()
        display(widgets.HTML(value=cards))

        display(widgets.VBox(
            [
                pil_to_widget_img(temp_humidity_img),
                pil_to_widget_img(pressure_img),
                pil_to_widget_img(users_img),
                pil_to_widget_img(temp_range_img),
                pil_to_widget_img(daylight_hours_img),
                pil_to_widget_img(sun_times_img)
            ],
            layout=widgets.Layout(
                align_items='center',
                justify_content='center',
                display='flex',
                flex_direction='column',
                gap='100px'
            )
        ))

refresh_button.on_click(refresh_dashboard)

##################~ End Graphs page ~###################
########################################################
########################################################


########################################################
########################################################
##################~ Start Admin page ~###################


# Create login credentials (username and password)
correct_username = "admin"
correct_password = "admin"

# Create widgets for entering the username and password
username_field = widgets.Text(
    description="Username:",
    placeholder="Enter username",
    layout=widgets.Layout(width='300px')
)

password_field = widgets.Password(
    description="Password:",
    placeholder="Enter password",
    layout=widgets.Layout(width='300px')
)


style_login_button = """
<style>
    .login-button {
        background-color: #4CAF50;
        color: white;
        border: none;
        padding: 8px 16px;
        font-size: 17px;
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s, transform 0.3s;
        margin: 0 auto;
        display: block;
    }

    .login-button:hover {
        background-color: #388E3C;
    }

    .button-container {
        display: flex;
        justify-content: center;
        align-items: center;
        gap: 15px;
        margin-top: 20px;
        margin-bottom: 20px;
    }
</style>
"""

display(widgets.HTML(value=style_login_button))

#login btn
login_btn = widgets.Button(
    description="Login",
    layout=widgets.Layout(width='300px')
)

login_btn.add_class("login-button")

#error message
login_error_message = widgets.Label(
    value="",
    style={'color': 'red'}
)



def create_index_inspector():
    # Create a text input widget for searching words in the index
    index_word_search_input = widgets.Text(
        placeholder='🔍 Search word in index...',
        layout=widgets.Layout(width='400px', height='40px')
    )

    # Create an output widget to display the indexed words
    index_word_box = widgets.Output(layout=widgets.Layout(
        border='1px solid #ccc',
        width='400px',
        height='300px',
        overflow_y='scroll',
        padding='10px',
        font_family='monospace'
    ))

    # Create a button to refresh the index
    refresh_index_btn = widgets.Button(
        description='🔁 Refresh Index',
        layout=widgets.Layout(width='180px')
    )

    # Create a button to export the words
    export_btn = widgets.Button(
        description='📤 Export Words',
        layout=widgets.Layout(width='180px'),
        button_style='info'
    )


    export_status = widgets.Label(value="")
    current_index_words = []
    stemmer = PorterStemmer()

    # Function to load indexed words from Firebase and store them in the current_index_words list
    def load_index_words():
        nonlocal current_index_words
        index_service.load_index_from_firebase()
        index = index_service.convert_index_for_firebase()
        current_index_words = sorted(index.keys())

    # Function to update the word box that displays the indexed words, based on the search input
    def update_index_word_box(change=None):
        search_text = index_word_search_input.value.strip()
        search_stem = stemmer.stem(search_text.lower()) if search_text else ""
        filtered = [word for word in current_index_words if search_stem in word]

        with index_word_box:
            # Clear the current output of the word box
            index_word_box.clear_output()

            # If there's a search text, print the search details
            if search_text:
                print(f'🔍 Search: "{search_text}" → Stem: "{search_stem}"')
                print('-' * 40)

            # Print each filtered word (those matching the search stem)
            for word in filtered:
                print(word)

    # Function to handle refreshing the index from Firebase and updating the word box
    def refresh_index_handler(btn):
        load_index_words()
        update_index_word_box()
        export_status.value = "Index refreshed from Firebase"

    # Function to export the indexed words to a text file
    def export_index_words_to_txt(btn):
        # If there are no words to export, update the status
        if not current_index_words:
            export_status.value = "No words to export."
            return

        try:
            file_path = "index_words_export.txt"
            with open(file_path, 'w', encoding='utf-8') as f:
                # Write each word in the current index to the file
                for word in current_index_words:
                    f.write(word + '\n')

            # Update the status to indicate the export is ready
            export_status.value = "Download..."
            # Trigger the file download
            files.download(file_path)
        except Exception as e:
            # In case of an error, update the status with the error message
            export_status.value = f"Error: {str(e)}"

    # Set up the search input observer to update the word box whenever the value changes
    index_word_search_input.observe(update_index_word_box, names='value')

    # Set up the refresh button click handler
    refresh_index_btn.on_click(refresh_index_handler)

    # Set up the export button click handler
    export_btn.on_click(export_index_words_to_txt)


    # Load the words initially
    load_index_words()
    update_index_word_box()

    #return the widgets with the results
    return widgets.VBox([
        widgets.HTML('<h4>📚 Indexed Words Overview</h4>'),
        index_word_search_input,
        index_word_box,
        widgets.HBox([refresh_index_btn, export_btn], layout=widgets.Layout(justify_content='center', gap='20px')),
        export_status
    ], layout=widgets.Layout(align_items='center', padding='20px'))







def show_admin_dashboard():
    with tab0_content:
        tab0_content.clear_output()

        # Display the logo widget at the center
        display(widgets.VBox([logo_widget], layout=widgets.Layout(align_items='center')))

        # Sample users' information
        users_info = [
            ["User1", "Active", "5 mins ago"],
            ["User2", "Inactive", "10 mins ago"],
            ["User3", "Active", "1 min ago"],
            ["User4", "Inactive", "30 mins ago"],
            ["User5", "Active", "15 mins ago"]
        ]

        # Construct the HTML table with user information
        table_html = "<table class='results-table' style='margin: auto;'>"
        table_html += "<tr><th>#</th><th>Username</th><th>Status</th><th>Last Activity</th></tr>"
        for i, user in enumerate(users_info, start=1):
            table_html += f"<tr><td>{i}</td><td>{user[0]}</td><td>{user[1]}</td><td>{user[2]}</td></tr>"
        table_html += "</table>"

        # Create a layout with the table, index inspector, and logout button
        content = widgets.VBox(
            [
                widgets.HTML(value=table_html),
                create_index_inspector(),  # Add an index inspector widget
                logout_btn  # Add a logout button
            ],
            layout=widgets.Layout(
                align_items='center',  # Center the content vertically
                justify_content='center',  # Center the content horizontally
                width='100%',  # Set width to 100%
                height='100%'  # Set height to 100%
            )
        )

        # Display the final content
        display(content)





# Function to perform login check
def handle_login_click(b):
    # Get the username and password entered
    entered_username = username_field.value
    entered_password = password_field.value

    # Check if the username and password are correct
    if entered_username == correct_username and entered_password == correct_password:
        # If login is correct, display the admin dashboard
        login_error_message.value = ""  # Clear the error message if login is correct
        show_admin_dashboard()
    else:
        # If login is incorrect, display an error message
        login_error_message.value = "Incorrect username or password. Please try again."

# Connect the login function to the button
login_btn.on_click(handle_login_click)

# CSS Styling for LOGOUT Button (with center alignment and reduction)
style_logout_button = """
<style>
  .logout-button {
      background-color: #D32F2F;
      color: white;
      border: none;
      padding: 8px 16px;
      font-size: 14px;
      border-radius: 8px;
      cursor: pointer;
      transition: background-color 0.3s, transform 0.3s;
      margin: 0 auto;
      display: block;
  }

  .logout-button:hover {
      background-color: #B71C1C;  /* Darker red when hovering over the button */
  }

  .button-container {
      display: flex;
      justify-content: center;
      align-items: center;
      gap: 15px;
      margin-top: 20px;
      margin-bottom: 20px;
  }
</style>
"""

display(widgets.HTML(value=style_logout_button))

logout_btn = widgets.Button(
    description="Logout",
    layout=widgets.Layout(width='150px', height='50px')  # גודל כפתור קטן יותר
)

logout_btn.add_class("logout-button")

def handle_logout_click(b):
    # Reset all values on the admin screen and return to the login page
    username_field.value = ''  # Reset the username field
    password_field.value = ''  # Reset the password field
    login_error_message.value = ''  # Clear the error message if there is one

    # Clear the content of the admin page
    tab0_content.clear_output(wait=True)

    # Display the login form again
    with tab0_content:
        display(widgets.VBox([  # Display the login form again
            logo_widget,
            username_field,
            password_field,
            login_btn,
            login_error_message
        ], layout=widgets.Layout(align_items='center', justify_content='center')))

    # Return to the login tab
    tabs.selected_index = 0  # This brings the user back to the login page

# Connect the function to the LOGOUT button
logout_btn.on_click(handle_logout_click)




##################~ End Admin page ~###################
########################################################
########################################################


#-------------------------------------------------------------------GAMING-------------------------------------------------------------------








# HTML style for tables and headers
display(HTML("""
<style>
.centered-table {
    margin-left: auto !important;
    margin-right: auto !important;
    text-align: center;
    direction: ltr;
    border-collapse: collapse;
    font-family: Arial, sans-serif;
    width: 60%;
    table-layout: fixed;
}
.gamification-title {
    text-align: center;
    font-size: 24px;
    font-weight: bold;
    margin-top: 20px;
}
.centered-table th, .centered-table td {
    padding: 12px;
    border: 1px solid #ccc;
    text-align: center;
    width: 33.33%;
    word-wrap: break-word;
}
h4 {
    text-align: center;
    font-family: Arial, sans-serif;
    margin-top: 30px;
}
hr {
    width: 60%;
    margin-left: auto;
    margin-right: auto;
}
</style>
"""))

# Initialize Firebase connection
FBconn = firebase.FirebaseApplication('https://cloud-e4313-default-rtdb.firebaseio.com/', None)

# Dictionary of example tasks to upload
tasks_to_upload = {
    # Easy tasks
    "task001": {"title": "Update README file", "difficulty": "easy", "points": 10, "complited": False},
    "task002": {"title": "Fix login bug", "difficulty": "easy", "points": 10, "complited": False},
    "task003": {"title": "Design homepage layout", "difficulty": "easy", "points": 10, "complited": False},
    "task004": {"title": "Add footer component", "difficulty": "easy", "points": 10, "complited": False},
    "task005": {"title": "Improve button styles", "difficulty": "easy", "points": 10, "complited": False},
    "task006": {"title": "Update contact info", "difficulty": "easy", "points": 10, "complited": False},
    "task007": {"title": "Correct typo in FAQ", "difficulty": "easy", "points": 10, "complited": False},
    "task008": {"title": "Resize logo", "difficulty": "easy", "points": 10, "complited": False},
    "task009": {"title": "Test login form", "difficulty": "easy", "points": 10, "complited": False},
    "task010": {"title": "Clean unused CSS", "difficulty": "easy", "points": 10, "complited": False},
    # Medium tasks
    "task011": {"title": "Build user profile page", "difficulty": "medium", "points": 30, "complited": False},
    "task012": {"title": "Integrate MongoDB", "difficulty": "medium", "points": 30, "complited": False},
    "task013": {"title": "Create search feature", "difficulty": "medium", "points": 30, "complited": False},
    "task014": {"title": "Add email verification", "difficulty": "medium", "points": 30, "complited": False},
    "task015": {"title": "Optimize image loading", "difficulty": "medium", "points": 30, "complited": False},
    "task016": {"title": "Create pagination system", "difficulty": "medium", "points": 30, "complited": False},
    "task017": {"title": "Add user avatar upload", "difficulty": "medium", "points": 30, "complited": False},
    "task018": {"title": "Filter table results", "difficulty": "medium", "points": 30, "complited": False},
    "task019": {"title": "Setup role-based access", "difficulty": "medium", "points": 30, "complited": False},
    "task020": {"title": "Add language selector", "difficulty": "medium", "points": 30, "complited": False},
    # Hard tasks
    "task021": {"title": "Automate test pipeline", "difficulty": "hard", "points": 50, "complited": False},
    "task022": {"title": "Implement dark mode toggle", "difficulty": "hard", "points": 50, "complited": False},
    "task023": {"title": "Deploy to Firebase hosting", "difficulty": "hard", "points": 50, "complited": False},
    "task024": {"title": "Create admin dashboard", "difficulty": "hard", "points": 50, "complited": False},
    "task025": {"title": "Configure CI/CD with GitLab", "difficulty": "hard", "points": 50, "complited": False},
    "task026": {"title": "Build analytics panel", "difficulty": "hard", "points": 50, "complited": False},
    "task027": {"title": "Secure API endpoints", "difficulty": "hard", "points": 50, "complited": False},
    "task028": {"title": "Real-time notifications", "difficulty": "hard", "points": 50, "complited": False},
    "task029": {"title": "Implement WebSocket chat", "difficulty": "hard", "points": 50, "complited": False},
    "task030": {"title": "Backup & restore feature", "difficulty": "hard", "points": 50, "complited": False},
}

# Upload taskBank to Firebase
for task_id, task_data in tasks_to_upload.items():
    FBconn.put('/gamification/taskBank/', task_id, task_data)

# Prepare tracking info
month = datetime.now().strftime('%Y-%m')
today = datetime.now().strftime('%Y-%m-%d')
difficulty_points = {"easy": 10, "medium": 30, "hard": 50}
firebase_users = FBconn.get('/gamification/users', None)
usernames = [name.capitalize() for name in firebase_users.keys()] if firebase_users else []

all_tasks = FBconn.get('/gamification/taskBank', None)


# Returns a set of task IDs completed by any user this month
def get_completed_task_ids():
    done_ids = set()
    for user in usernames:
        user_tasks = FBconn.get(f'/gamification/taskBank/{month}/{user}', None)
        if user_tasks:
            for task_id, task in user_tasks.items():
                if task.get("completed", False):
                    done_ids.add(task_id)
    return done_ids


# Calculates daily and monthly scores for all users from Firebase
def calculate_scores():
    daily_scores = []
    monthly_scores = []
    for user in usernames:
        tasks = FBconn.get(f'/gamification/tasks/{month}/{user}', None)
        daily_score = 0
        monthly_score = 0
        if tasks:
            for task_id, task_data in tasks.items():
                diff = task_data.get("difficulty", "easy")
                completed = task_data.get("completed", False)
                date = task_data.get("date", "")
                points = difficulty_points.get(diff, 10)
                if completed:
                    monthly_score += points
                    if date == today:
                        daily_score += points
        daily_scores.append({"Name": user, "Daily Score": daily_score})
        monthly_scores.append({"Name": user, "Monthly Score": monthly_score})
    return pd.DataFrame(daily_scores), pd.DataFrame(monthly_scores)


# Highlights the max and min value in a given column of a DataFrame
def highlight_top_bottom(s, column):
    max_idx = s[column].idxmax()
    min_idx = s[column].idxmin()
    style = pd.DataFrame('', index=s.index, columns=s.columns)
    style.loc[max_idx, column] = 'background-color: lightgreen'
    style.loc[min_idx, column] = 'background-color: lightcoral'
    return style


# Sorts and formats a score table with rank and custom styles
def display_ranked_table(df, score_col):
    df_sorted = df.sort_values(by=score_col, ascending=False).copy()
    df_sorted['Rank'] = range(1, len(df_sorted) + 1)
    df_sorted = df_sorted[[score_col, 'Name', 'Rank']]
    return df_sorted.style \
        .apply(highlight_top_bottom, column=score_col, axis=None) \
        .set_table_styles([
            {'selector': 'th', 'props': [('padding', '12px'), ('border', '1px solid #999')]},
            {'selector': 'td', 'props': [('padding', '10px'), ('border', '1px solid #ccc')]}
        ]) \
        .set_table_attributes('class="centered-table"') \
        .hide(axis='index')


# Create widgets for display
df_daily, df_monthly = calculate_scores()
out_daily = widgets.Output()
out_monthly = widgets.Output()

# Render and display the daily score table inside the output widget
with out_daily:
    out_daily.clear_output()
    display(HTML("<h4>📆 Daily Score Table</h4><div style='display:flex; justify-content:center;'>"))
    html_daily = display_ranked_table(df_daily, 'Daily Score').to_html()
    display(HTML(html_daily))
    display(HTML("</div>"))

# Render and display the monthly score table inside the output widget
with out_monthly:
    out_monthly.clear_output()
    display(HTML("<h4>📅 Monthly Score Table</h4><div style='display:flex; justify-content:center;'>"))
    html_monthly = display_ranked_table(df_monthly, 'Monthly Score').to_html()
    display(HTML(html_monthly))
    display(HTML("</div>"))


# User and task selection widgets
user_dropdown = widgets.Dropdown(options=usernames, description='Select User:')
difficulty_dropdown = widgets.Dropdown(
    options=[('Easy (10)', 'easy'), ('Medium (30)', 'medium'), ('Hard (50)', 'hard')],
    description='Difficulty:'
)
task_dropdown = widgets.Dropdown(options=[], description='Task:')
assign_btn = widgets.Button(description='✅ Complete Task', button_style='success')
assign_msg = widgets.Output()
task_data_map = {}


# Updates the task dropdown based on selected difficulty.
# Also resets daily tasks and deletes yesterday's completed ones if needed.
def update_tasks_list(change=None):
    global all_tasks
    all_tasks = FBconn.get('/gamification/taskBank', None)

    # Daily reset: make all tasks available again + remove yesterday's tasks
    last_reset_date = FBconn.get('/gamification/lastDailyReset', None)
    if last_reset_date != today:
        for task_id in all_tasks:
            FBconn.patch(f'/gamification/taskBank/{task_id}', {'complited': False})
        FBconn.put('/gamification', 'lastDailyReset', today)

        yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
        for user in usernames:
            user_tasks = FBconn.get(f'/gamification/tasks/{month}/{user}', None)
            if user_tasks:
                for task_id, task in user_tasks.items():
                    if task.get("date") == yesterday:
                        FBconn.delete(f'/gamification/tasks/{month}/{user}', task_id)

    # Filter and display tasks by selected difficulty
    selected_diff = difficulty_dropdown.value
    task_dropdown.options = []
    task_data_map.clear()
    options = []

    if all_tasks:
        for task_id, task in all_tasks.items():
            if isinstance(task, dict) and task.get("difficulty") == selected_diff and not task.get("complited", False):
                title = task.get("title", f"Task {task_id}")
                options.append((title, task_id))
                task_data_map[task_id] = task

    task_dropdown.options = options if options else [("No tasks available", None)]




# Assigns a selected task to the chosen user, updates Firebase and refreshes scores.
def assign_task(btn):

    # Get selected user and task from the dropdowns
    selected_user = user_dropdown.value
    selected_task_id = task_dropdown.value

    # If no valid task is selected, show a warning message
    if not selected_task_id or selected_task_id not in task_data_map:
        with assign_msg:
            assign_msg.clear_output()
            print("⚠️ No valid task selected.")
        return

    # Copy the selected task data and mark it as completed for today
    task_data = task_data_map[selected_task_id].copy()
    task_data['completed'] = True
    task_data['date'] = today

    # Save the completed task under the user's monthly path in Firebase
    FBconn.patch(f'/gamification/tasks/{month}/{selected_user}/{selected_task_id}', task_data)

    # Mark the task as unavailable globally in taskBank
    FBconn.patch(f'/gamification/taskBank/{selected_task_id}', {'complited': True})

    # Recalculate daily and monthly scores after task completion
    global df_daily, df_monthly
    df_daily, df_monthly = calculate_scores()

    # Render and update daily score table
    with out_daily:
        out_daily.clear_output()
        display(HTML("<h4>📆 Daily Score Table</h4><div style='display:flex; justify-content:center;'>"))
        display(HTML(display_ranked_table(df_daily, 'Daily Score').to_html()))
        display(HTML("</div>"))

    # Render and update monthly score table
    with out_monthly:
        out_monthly.clear_output()
        display(HTML("<h4>📅 Monthly Score Table</h4><div style='display:flex; justify-content:center;'>"))
        display(HTML(display_ranked_table(df_monthly, 'Monthly Score').to_html()))
        display(HTML("</div>"))

    # Clear the user message area (to prepare for a new message)
    with assign_msg:
        assign_msg.clear_output()


    update_tasks_list()

assign_btn.on_click(assign_task)
difficulty_dropdown.observe(update_tasks_list, names='value')

# Initialize display
update_tasks_list()


############### End Gaming ######################
########################################################
########################################################



########################################################
########################################################
# Tabs

# Display admin tab
tab0_content = widgets.Output()
with tab0_content:
    username_field = widgets.Text(
        description="Username:",
        placeholder="Enter username",
        layout=widgets.Layout(width='300px')
    )

    password_field = widgets.Password(
        description="Password:",
        placeholder="Enter password",
        layout=widgets.Layout(width='300px')
    )

    login_btn = widgets.Button(
        description="Login",
        layout=widgets.Layout(width='300px' , height='50px')
    )

    login_btn.add_class("login-button")

    login_error_message = widgets.Label(
        value="",
        style={'color': 'red'}
    )

    def handle_login_click(b):
        entered_username = username_field.value
        entered_password = password_field.value

        if entered_username == "admin" and entered_password == "admin":
            login_error_message.value = ""
            show_admin_dashboard()
        else:
            login_error_message.value = "Incorrect username or password. Please try again."

    login_btn.on_click(handle_login_click)

    display(widgets.VBox([
        logo_widget,
        username_field,
        password_field,
        login_btn,
        login_error_message
    ], layout=widgets.Layout(align_items='center', justify_content='center')))



# Display graphs tab
with tab1_content:
    display(widgets.VBox(
        [logo_widget, refresh_button],
        layout=widgets.Layout(align_items='center', justify_content='center', gap='20px')
    ))
    cards, temp_humidity_img, pressure_img, users_img, temp_range_img, daylight_hours_img, sun_times_img = update_dashboard()
    display(widgets.HTML(value=cards))

    display(widgets.VBox(
    [
        pil_to_widget_img(temp_humidity_img),
        pil_to_widget_img(pressure_img),
        pil_to_widget_img(users_img),
        pil_to_widget_img(temp_range_img),
        pil_to_widget_img(daylight_hours_img),
        pil_to_widget_img(sun_times_img)
    ],
    layout=widgets.Layout(
        align_items='center',
        justify_content='center',
        display='flex',
        flex_direction='column',
        gap='100px'
    )
))

# Display gamification tab
tab2_content = widgets.VBox([logo_widget,
    widgets.HTML('<h3 class="gamification-title">🎮 Gamification - Employee Scoring</h3>'),
    widgets.HBox([user_dropdown, difficulty_dropdown, task_dropdown, assign_btn],
                 layout=widgets.Layout(justify_content='flex-end', padding='10px')),
    assign_msg,
    widgets.HTML("<hr>"),
    out_daily,
    widgets.HTML("<hr>"),
    out_monthly
], layout=widgets.Layout(padding='30px', border='1px solid #ccc', align_items='center'))

# Display search tab
tab3_content = widgets.Output()
with tab3_content:
    display(main_container)

tab4_content = widgets.Output()
with tab4_content:
    display(widgets.VBox([logo_widget, searchResults],
                         layout=widgets.Layout(align_items='center')))


tabs = widgets.Tab(children=[tab0_content,tab1_content,tab2_content,tab3_content],
                   layout=widgets.Layout(width='1200px', height='800px'))
tabs.set_title(0, 'Admin')
tabs.set_title(1, 'Graphs')
tabs.set_title(2, 'Gamification')
tabs.set_title(3, 'Search')


display(tabs)


text = """
cloud computing and machine learning &nbsp;&nbsp;&nbsp;&nbsp; the best cloud computing platform &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2025 cloud computing trends<br>
top 10 mqtt brokers &nbsp;&nbsp;&nbsp;&nbsp; cloud computing introduction for beginners &nbsp;&nbsp;&nbsp;&nbsp; data science using python &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;machine learning basics
"""

html_content = f"""
    <p style="color: #C0C0C0; padding: 10px; font-size: 10px;">{text}</p>
"""

display(widgets.HTML(value=html_content))




HTML(value='\n<style>\n    html, body {\n        background-color: #f4f4f4;\n        margin: 0;\n        heigh…

HTML(value='\n<style>\n    .login-button {\n        background-color: #4CAF50; \n        color: white;\n      …

HTML(value='\n<style>\n  .logout-button {\n      background-color: #D32F2F;\n      color: white;\n      border…

HTML(value='\n    <p style="color: #C0C0C0; padding: 10px; font-size: 10px;">\ncloud computing and machine lea…